# Training des ML-Algorithmus

## Laden

In [1]:
import csv
import numpy as np

#time.sleep(60*60*14)



with open('combined_malicious_and_normal.csv', 'r') as csvfile:
    csvtext = csvfile.readlines()

X = []
for line in csvtext:
    X.append(tuple(line.strip().split(', '))[0])

## Ergebnisse wieder anfügen für jeden Eintrag der Liste

In [2]:
#Der erste Eintrag der kombinierten Liste ist maliziös. Ab da an alternierend.
#0 = malicious
#1 = good
y = []
for i in range(len(X)):
    if(i%2 == 0):
        y.append(0)
    else:
        y.append(1)
print(y[:3])

[0, 1, 0]


## Datensatz aufteilen

In [3]:
import copy

def filter_numbers(data):
    function_data = copy.copy(data)
    function_data = [item.replace('0', '1') for item in function_data]
    function_data = [item.replace('2', '1') for item in function_data]
    function_data = [item.replace('3', '1') for item in function_data]
    function_data = [item.replace('4', '1') for item in function_data]
    function_data = [item.replace('5', '1') for item in function_data]
    function_data = [item.replace('6', '1') for item in function_data]
    function_data = [item.replace('7', '1') for item in function_data]
    function_data = [item.replace('8', '1') for item in function_data]
    function_data = [item.replace('9', '1') for item in function_data]
    return function_data

max_count = 200000
del X[max_count:]
del y[max_count:]
#print(X[:5])
X_Raw = filter_numbers(X)
#print(X_Raw[:5])
#print(X[:5])
#print(str(len(X_Raw)))
del X

## URL's vektorisieren

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
import time

def vectorize(n_gram_min, n_gram_max, data, amount_of_params):
    #dafür sorgen dass der Ursprungsdatensatz nicht korrumpiert wird
    data = copy.deepcopy(data)

    #Initialisieren und Dictionary anlegen
    vectorizer = CountVectorizer(analyzer='word', token_pattern=r'[^\s+]', max_features = amount_of_params, ngram_range=(n_gram_min, n_gram_max))
    start_time_fit = time.time()
    vectorizer.fit(data)
    end_time_fit = time.time()
    time_for_fit = end_time_fit - start_time_fit
    
    #Dictionary anwenden und N-Gram Vektorisierng durchführen
    start_time_transform = time.time()
    data = vectorizer.transform(data)
    end_time_transform = time.time()
    time_for_transform = end_time_transform - start_time_transform
    
    #Bisschen RAM freigeben
    del vectorizer
    print("time_for_fit:" + str(time_for_fit))
    print("time_for_transform:" + str(time_for_transform))   
    return data

# Model definieren und trainieren

In [5]:
from sklearn.ensemble import RandomForestClassifier

#ML-Modell-Erstellung
def create_and_fit_model(X_train,y_train,tree_count):
    classifier = RandomForestClassifier(n_estimators=tree_count, random_state=0,n_jobs=-1)
    classifier.fit(X_train, y_train) 
    return classifier

# Parameter testen und dokumentieren

In [6]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import time
import random
from sklearn.model_selection import train_test_split

#print(classification_report(y_test,y_pred))
def documentation_of_results(n_gram_min,n_gram_max,accuracy,TP,TN,FP,FN,parameter_count,tree_count, allowed_feature_count,test_size_percentage,vectorize_time,modelcreation_time, modelprediction_time, time_test, max_count):
    with open('results_test_parameter_und_zeit_mit_mehr_daten.txt', "a") as fhandle:
        fhandle.write(f'N-Grams-Range: (' + str(n_gram_min) + '/' + str(n_gram_max) 
                      + '); Accuracy: ' + str(accuracy) 
                      + '; TP: ' + str(TP) 
                      + '; TN: '+ str(TN) 
                      + '; FP: '+ str(FP) 
                      + '; FN: ' + str(FN) 
                      + '; Max-Allowed-Parameter-Count: ' + str(allowed_feature_count) 
                      + '; Estimator-Count: ' + str(tree_count)
                      + '; Used-Parameter-Count: ' + str(parameter_count)                    
                      + '; Test-Size-Percentage: ' + str(test_size_percentage)   
                      + '; Vectorize-Time: ' + str(vectorize_time)
                      + '; Model-Creation-Time: ' + str(modelcreation_time)                    
                      + '; Model-Prediction-Time: ' + str(modelprediction_time)          
                      + '; Model-Time-Test: ' + str(time_test)    
                      + '; Total-Count-Of-Entries: ' + str(max_count)                          
                      #+ '; Vectorize-Time per Entry: ' + str(int(vectorize_time/max_count))
                      #+ '; Model-Creation-Time per Entry: ' + str(int(modelcreation_time/(max_count*(1-test_size_percentage))))                    
                      #+ '; Model-Prediction-Time per Entry: ' + str(int(modelprediction_time/(max_count*test_size_percentage))) 
                      + '\n')

#Kombinationen testen
for x in range(1000):
    np.random.seed(31415)
    print("----" + str(x) + "-te Runde-----")
    test_size_percentage = random.randint(1,80)/100
    print("Test-Size-Dataset-Percentage:" + str(test_size_percentage))  
    tree_count = random.randint(1,10)
    print("Tree-Count:" + str(tree_count))
    allowed_feature_count = random.randint(1,800)
    print("Feature-Count:" + str(allowed_feature_count))
    n_gram_max = random.randint(0,10)
    print("N-Gram-Max:" + str(n_gram_max))
    n_gram_min = random.randint(n_gram_max-5,n_gram_max)
    print("N-Gram-Min:" + str(n_gram_min))
    
    #test time calculation
    start_time_test = time.time()
    z = 0
    for i in range (10000000):
        z+=1
    end_time_test = time.time()
    time_test = end_time_test - start_time_test 
    print("Time-Test:" + str(time_test))   
    del z
    #end timetest
    
    start_vector = time.time()
    X = vectorize(n_gram_min, n_gram_max, X_Raw,allowed_feature_count)
    ende_vector = time.time()
    vectorize_time = '{:5.3f}s'.format(ende_vector-start_vector)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_percentage, random_state=0)
    start_modelcreation = time.time()
    classifier = create_and_fit_model(X_train,y_train, tree_count)
    ende_modelcreation = time.time()
    modelcreation_time = '{:5.3f}s'.format(ende_modelcreation-start_modelcreation)
    start_modelprediction = time.time()
    y_pred = classifier.predict(X_test)
    ende_modelprediction = time.time()
    modelprediction_time = '{:5.3f}s'.format(ende_modelprediction-start_modelprediction)
    print("y_train:" + str(len(y_train)))    
    print("y_test:" + str(len(y_test)) )
    
    print("Vectorize-Time:" + str(vectorize_time))
    print("Model-Creation-Time:" + str(modelcreation_time))  
    print("Model-Prediction-Time:" + str(modelprediction_time))    

    accuracy = accuracy_score(y_test, y_pred)
    print("accuracy:" + str(accuracy))
    
    CM = confusion_matrix(y_test, y_pred)
    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    print("---------")
    

    documentation_of_results(n_gram_min,n_gram_max,accuracy,TP,TN,FP,FN,allowed_feature_count,tree_count, allowed_feature_count,test_size_percentage,vectorize_time,modelcreation_time, modelprediction_time,time_test, max_count)  
    
    #Free up memory space
    del X,  X_train,X_test, y_train, y_test, classifier, y_pred, accuracy, CM

----0-te Runde-----
Test-Size-Dataset-Percentage:0.26
Tree-Count:10
Feature-Count:708
N-Gram-Max:7
N-Gram-Min:4
Time-Test:0.870537519454956
time_for_fit:18.870010137557983
time_for_transform:8.449985980987549
y_train:148000
y_test:52000
Vectorize-Time:27.626s
Model-Creation-Time:6.178s
Model-Prediction-Time:0.129s
accuracy:0.7642692307692308
---------
----1-te Runde-----
Test-Size-Dataset-Percentage:0.42
Tree-Count:6
Feature-Count:800
N-Gram-Max:2
N-Gram-Min:0
Time-Test:1.0243000984191895
time_for_fit:5.631381034851074
time_for_transform:6.469816446304321
y_train:116000
y_test:84000
Vectorize-Time:12.207s
Model-Creation-Time:5.115s
Model-Prediction-Time:0.140s
accuracy:0.8294880952380952
---------
----2-te Runde-----
Test-Size-Dataset-Percentage:0.39
Tree-Count:7
Feature-Count:298
N-Gram-Max:8
N-Gram-Min:4
Time-Test:1.1482512950897217
time_for_fit:24.355012893676758
time_for_transform:10.562586545944214
y_train:122000
y_test:78000
Vectorize-Time:35.344s
Model-Creation-Time:1.259s
Model

Time-Test:1.2232961654663086
time_for_fit:20.80019450187683
time_for_transform:7.267271518707275
y_train:178000
y_test:22000
Vectorize-Time:28.472s
Model-Creation-Time:0.791s
Model-Prediction-Time:0.130s
accuracy:0.6434545454545455
---------
----24-te Runde-----
Test-Size-Dataset-Percentage:0.28
Tree-Count:10
Feature-Count:83
N-Gram-Max:3
N-Gram-Min:-2
Time-Test:1.1812398433685303
time_for_fit:13.610179662704468
time_for_transform:10.652499675750732
y_train:143999
y_test:56001
Vectorize-Time:24.407s
Model-Creation-Time:13.428s
Model-Prediction-Time:0.134s
accuracy:0.8303780289637686
---------
----25-te Runde-----
Test-Size-Dataset-Percentage:0.69
Tree-Count:8
Feature-Count:381
N-Gram-Max:8
N-Gram-Min:8
Time-Test:1.1499502658843994
time_for_fit:6.302703619003296
time_for_transform:2.5568463802337646
y_train:62000
y_test:138000
Vectorize-Time:9.050s
Model-Creation-Time:0.133s
Model-Prediction-Time:0.107s
accuracy:0.6469492753623188
---------
----26-te Runde-----
Test-Size-Dataset-Percent

y_train:40000
y_test:160000
Vectorize-Time:38.784s
Model-Creation-Time:0.222s
Model-Prediction-Time:0.075s
accuracy:0.74135625
---------
----47-te Runde-----
Test-Size-Dataset-Percentage:0.25
Tree-Count:4
Feature-Count:390
N-Gram-Max:1
N-Gram-Min:1
Time-Test:1.1601576805114746
time_for_fit:1.843196153640747
time_for_transform:1.847095012664795
y_train:150000
y_test:50000
Vectorize-Time:3.792s
Model-Creation-Time:3.489s
Model-Prediction-Time:0.125s
accuracy:0.77912
---------
----48-te Runde-----
Test-Size-Dataset-Percentage:0.01
Tree-Count:3
Feature-Count:614
N-Gram-Max:10
N-Gram-Min:5
Time-Test:1.1505532264709473
time_for_fit:30.370322704315186
time_for_transform:10.606470823287964
y_train:198000
y_test:2000
Vectorize-Time:41.606s
Model-Creation-Time:1.045s
Model-Prediction-Time:0.110s
accuracy:0.6695
---------
----49-te Runde-----
Test-Size-Dataset-Percentage:0.79
Tree-Count:8
Feature-Count:53
N-Gram-Max:8
N-Gram-Min:6
Time-Test:1.183332920074463
time_for_fit:16.974790334701538
time_f

Time-Test:1.3805649280548096
time_for_fit:9.984584093093872
time_for_transform:4.913592100143433
y_train:84000
y_test:116000
Vectorize-Time:15.120s
Model-Creation-Time:0.454s
Model-Prediction-Time:0.120s
accuracy:0.7132931034482759
---------
----71-te Runde-----
Test-Size-Dataset-Percentage:0.55
Tree-Count:8
Feature-Count:98
N-Gram-Max:9
N-Gram-Min:8
Time-Test:1.1419553756713867
time_for_fit:11.556876182556152
time_for_transform:3.856163501739502
y_train:89999
y_test:110001
Vectorize-Time:15.681s
Model-Creation-Time:0.120s
Model-Prediction-Time:0.112s
accuracy:0.5743947782292889
---------
----72-te Runde-----
Test-Size-Dataset-Percentage:0.48
Tree-Count:1
Feature-Count:155
N-Gram-Max:5
N-Gram-Min:5
Time-Test:1.4475760459899902
time_for_fit:5.021760940551758
time_for_transform:2.780804395675659
y_train:104000
y_test:96000
Vectorize-Time:8.005s
Model-Creation-Time:0.233s
Model-Prediction-Time:0.047s
accuracy:0.6650833333333334
---------
----73-te Runde-----
Test-Size-Dataset-Percentage:0

y_train:46000
y_test:154000
Vectorize-Time:15.655s
Model-Creation-Time:0.111s
Model-Prediction-Time:0.110s
accuracy:0.6120324675324675
---------
----94-te Runde-----
Test-Size-Dataset-Percentage:0.51
Tree-Count:7
Feature-Count:356
N-Gram-Max:1
N-Gram-Min:0
Time-Test:1.1502201557159424
time_for_fit:1.8334739208221436
time_for_transform:1.8485517501831055
y_train:98000
y_test:102000
Vectorize-Time:3.790s
Model-Creation-Time:2.411s
Model-Prediction-Time:0.142s
accuracy:0.8188333333333333
---------
----95-te Runde-----
Test-Size-Dataset-Percentage:0.16
Tree-Count:1
Feature-Count:322
N-Gram-Max:1
N-Gram-Min:-3
Time-Test:1.2257730960845947
time_for_fit:1.8517811298370361
time_for_transform:1.9080195426940918
y_train:168000
y_test:32000
Vectorize-Time:3.869s
Model-Creation-Time:3.264s
Model-Prediction-Time:0.038s
accuracy:0.7615
---------
----96-te Runde-----
Test-Size-Dataset-Percentage:0.44
Tree-Count:4
Feature-Count:126
N-Gram-Max:4
N-Gram-Min:0
Time-Test:1.2099690437316895
time_for_fit:10

...

y_train:194000
y_test:6000
Vectorize-Time:23.863s
Model-Creation-Time:34.650s
Model-Prediction-Time:0.125s
accuracy:0.846
---------
----893-te Runde-----
Test-Size-Dataset-Percentage:0.2
Tree-Count:2
Feature-Count:424
N-Gram-Max:0
N-Gram-Min:-3
Time-Test:1.1849479675292969
time_for_fit:10.06038236618042
time_for_transform:6.384868860244751
y_train:160000
y_test:40000
Vectorize-Time:16.635s
Model-Creation-Time:0.241s
Model-Prediction-Time:0.116s
accuracy:0.74605
---------
----894-te Runde-----
Test-Size-Dataset-Percentage:0.44
Tree-Count:4
Feature-Count:501
N-Gram-Max:8
N-Gram-Min:5
Time-Test:1.1516594886779785
time_for_fit:21.03621220588684
time_for_transform:7.322768926620483
y_train:112000
y_test:88000
Vectorize-Time:28.778s
Model-Creation-Time:0.595s
Model-Prediction-Time:0.125s
accuracy:0.6837159090909091
---------
----895-te Runde-----
Test-Size-Dataset-Percentage:0.42
Tree-Count:6
Feature-Count:583
N-Gram-Max:2
N-Gram-Min:-3
Time-Test:1.1772925853729248
time_for_fit:13.3602101802

Time-Test:1.167785406112671
time_for_fit:16.309256076812744
time_for_transform:4.961002349853516
y_train:128000
y_test:72000
Vectorize-Time:21.650s
Model-Creation-Time:0.365s
Model-Prediction-Time:0.125s
accuracy:0.6156111111111111
---------
----917-te Runde-----
Test-Size-Dataset-Percentage:0.77
Tree-Count:6
Feature-Count:162
N-Gram-Max:7
N-Gram-Min:4
Time-Test:1.150244951248169
time_for_fit:18.506153345108032
time_for_transform:8.023857355117798
y_train:46000
y_test:154000
Vectorize-Time:26.833s
Model-Creation-Time:0.128s
Model-Prediction-Time:0.130s
accuracy:0.7100974025974026
---------
----918-te Runde-----
Test-Size-Dataset-Percentage:0.29
Tree-Count:7
Feature-Count:740
N-Gram-Max:2
N-Gram-Min:0
Time-Test:1.1474931240081787
time_for_fit:5.628353595733643
time_for_transform:5.713198661804199
y_train:142000
y_test:58000
Vectorize-Time:11.454s
Model-Creation-Time:10.815s
Model-Prediction-Time:0.135s
accuracy:0.8450689655172414
---------
----919-te Runde-----
Test-Size-Dataset-Percent

y_train:128000
y_test:72000
Vectorize-Time:38.999s
Model-Creation-Time:4.176s
Model-Prediction-Time:0.130s
accuracy:0.7654027777777778
---------
----940-te Runde-----
Test-Size-Dataset-Percentage:0.62
Tree-Count:7
Feature-Count:167
N-Gram-Max:9
N-Gram-Min:8
Time-Test:1.1514909267425537
time_for_fit:11.482739925384521
time_for_transform:3.8400049209594727
y_train:76000
y_test:124000
Vectorize-Time:15.608s
Model-Creation-Time:0.125s
Model-Prediction-Time:0.120s
accuracy:0.5938709677419355
---------
----941-te Runde-----
Test-Size-Dataset-Percentage:0.01
Tree-Count:4
Feature-Count:370
N-Gram-Max:7
N-Gram-Min:6
Time-Test:1.1442713737487793
time_for_fit:11.07991647720337
time_for_transform:4.240208387374878
y_train:198000
y_test:2000
Vectorize-Time:15.569s
Model-Creation-Time:0.906s
Model-Prediction-Time:0.115s
accuracy:0.6285
---------
----942-te Runde-----
Test-Size-Dataset-Percentage:0.62
Tree-Count:7
Feature-Count:43
N-Gram-Max:10
N-Gram-Min:10
Time-Test:1.2858154773712158
time_for_fit:

Time-Test:1.2848613262176514
time_for_fit:19.154807090759277
time_for_transform:8.008421659469604
y_train:46000
y_test:154000
Vectorize-Time:27.479s
Model-Creation-Time:0.480s
Model-Prediction-Time:0.255s
accuracy:0.7363051948051949
---------
----964-te Runde-----
Test-Size-Dataset-Percentage:0.64
Tree-Count:6
Feature-Count:366
N-Gram-Max:7
N-Gram-Min:6
Time-Test:1.2070493698120117
time_for_fit:11.062938451766968
time_for_transform:4.602288246154785
y_train:72000
y_test:128000
Vectorize-Time:15.930s
Model-Creation-Time:0.225s
Model-Prediction-Time:0.115s
accuracy:0.6462421875
---------
----965-te Runde-----
Test-Size-Dataset-Percentage:0.4
Tree-Count:8
Feature-Count:200
N-Gram-Max:5
N-Gram-Min:3
Time-Test:1.1546423435211182
time_for_fit:10.38673448562622
time_for_transform:6.14009952545166
y_train:120000
y_test:80000
Vectorize-Time:16.698s
Model-Creation-Time:2.710s
Model-Prediction-Time:0.145s
accuracy:0.7661125
---------
----966-te Runde-----
Test-Size-Dataset-Percentage:0.24
Tree-Co

Time-Test:1.1748390197753906
time_for_fit:8.055434703826904
time_for_transform:8.98435115814209
y_train:124000
y_test:76000
Vectorize-Time:17.152s
Model-Creation-Time:4.047s
Model-Prediction-Time:0.130s
accuracy:0.8226973684210527
---------
----988-te Runde-----
Test-Size-Dataset-Percentage:0.3
Tree-Count:10
Feature-Count:306
N-Gram-Max:0
N-Gram-Min:0
Time-Test:1.2000157833099365
time_for_fit:2.1502532958984375
time_for_transform:2.1350464820861816
y_train:140000
y_test:60000
Vectorize-Time:4.395s
Model-Creation-Time:0.495s
Model-Prediction-Time:0.120s
accuracy:0.71545
---------
----989-te Runde-----
Test-Size-Dataset-Percentage:0.1
Tree-Count:4
Feature-Count:166
N-Gram-Max:1
N-Gram-Min:-4
Time-Test:1.1751527786254883
time_for_fit:1.8657281398773193
time_for_transform:1.8772337436676025
y_train:180000
y_test:20000
Vectorize-Time:3.854s
Model-Creation-Time:4.945s
Model-Prediction-Time:0.110s
accuracy:0.77865
---------
----990-te Runde-----
Test-Size-Dataset-Percentage:0.7
Tree-Count:7
F